In [1]:
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_binary
from chromedriver_binary import utils
import re
import os
import time


In [162]:
# options = Options()
# preference = {'download.default_directory': os.path.join(os.getcwd(),"data/citations"), "safebrowsing.enabled": "false"}
# options.add_experimental_option('prefs', preference)
driver = webdriver.Chrome()
# driver = webdriver.Chrome(options=options)
# driver.set_window_size(2000,1500)
wait = WebDriverWait(driver, 1000)

In [195]:
driver.get("https://pwr.edu.pl/pracownicy/strefa-pracownika/wyszukiwarka-pracownikow")


In [196]:
def get_array_of_jednostka_names(element):
    innerHTML = element.get_attribute('innerHTML')

    innerHTML = re.sub(" <i aria-hidden=\"true\" class=\"fa fa-angle-right\"></i> ", ">",innerHTML)
    innerHTML = re.sub("\t", "", innerHTML)
    innerHTML = re.sub("\n","",innerHTML)
    innerHTML = re.sub("<a.*\">", "", innerHTML)
    innerHTML = re.sub("</a>", "", innerHTML)
    innerHTML = re.sub("</a>", "", innerHTML)

    return innerHTML.split(">")

def trim_science_titles(name):
    trimmed = name.replace("mgr ", "")
    trimmed = trimmed.replace("inż. ", "")
    trimmed = trimmed.replace("dr ", "")
    trimmed = trimmed.replace("prof. ", "")
    trimmed = trimmed.replace("hab. ", "")
    trimmed = trimmed.replace("arch. ", "")
    return trimmed

In [197]:
list_of_employees = list()
for letter in range(1,28):
    driver.find_element(By.CSS_SELECTOR, f".letter:nth-child({letter})").click()

    for page in range(2,100):
        for idx in range(1,11):
            for i in range(1,10):
                try:
                    text = driver.find_element(By.CSS_SELECTOR, f".search-workers-result:nth-child({idx}) tr:nth-child({i})").text
                    if text.startswith("Jednostka:"):
                        element = driver.find_element(By.CSS_SELECTOR, f".search-workers-result:nth-child({idx}) tr:nth-child({i}) > td:nth-child(2)")
                        name = driver.find_element(By.CSS_SELECTOR, f".search-workers-result:nth-child({idx}) tr:nth-child(1)").text
                        name = trim_science_titles(name)
                        departaments = get_array_of_jednostka_names(element)
                        if departaments[0].startswith("Wydział") and departaments[1].startswith("Katedra"):
                            list_of_employees.append((name, departaments[0], departaments[1]))
                        break
                except:
                    break

        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        try:
            page_button =driver.find_element (By.LINK_TEXT, f"{page}")
            page_button.click()
        except:
            break

    driver.execute_script("window.scrollTo(0,0)")




In [212]:
list_of_employees[2161] = ('Oleksan Yeroshkin', list_of_employees[2161][1], list_of_employees[2161][2])

('Oleksan Yeroshkin',
 'Wydział Informatyki i Zarządzania',
 'Katedra Informatyki i Inżynierii Systemów')

In [228]:
df = pd.DataFrame(columns=['first_name','last_name', 'wydzial', 'katedra'])
for employee in list_of_employees:
    name = employee[0]
    wydzial = employee[1]
    katedra = employee[2]
    names = name.split(" ")
    df = df.append({'first_name': names[0], 'last_name': names[1], 'wydzial': wydzial, 'katedra': katedra}, ignore_index=True)

Katedra Mechaniki Budowli i Inżynierii Miejskiej
Katedra Teorii Pola, Układów Elektronicznych i Optoelektroniki
Katedra Górnictwa i Geodezji
Katedra Metrologii Elektronicznej i Fotonicznej
Katedra Maszyn, Napędów i Pomiarów Elektrycznych
Katedra Inżynierii Ochrony Środowiska
Katedra Inżynierii Ochrony Środowiska
Katedra Fizyki Doświadczalnej
Katedra Chemii i Technologii Paliw
Katedra Architektury Użyteczności Publicznej, Podstaw Projektowania  i Kształtowania Środowiska
Katedra Inżynierii Pojazdów
Katedra Obróbki Plastycznej, Spawalnictwa i Metrologii
Katedra Inżynierii i Modelowania Materiałów Zaawansowanych
Katedra Techniki Cieplnej
Katedra Inżynierii Pojazdów
Katedra Telekomunikacji i Teleinformatyki
Katedra Urbanistyki i Procesów Osadniczych
Katedra Fizyki Doświadczalnej
Katedra Klimatyzacji, Ogrzewnictwa, Gazownictwa i Ochrony Powietrza
Katedra Mechaniki, Inżynierii Materiałowej i Biomedycznej
Katedra Maszyn, Napędów i Pomiarów Elektrycznych
Katedra Podstaw Konstrukcji Maszyn i Uk

In [221]:
df['wydzial'].unique()

array(['Wydział Budownictwa Lądowego i Wodnego', 'Wydział Elektroniki',
       'Wydział Geoinżynierii, Górnictwa i Geologii',
       'Wydział Elektryczny', 'Wydział Inżynierii Środowiska',
       'Wydział Podstawowych Problemów Techniki', 'Wydział Chemiczny',
       'Wydział Architektury', 'Wydział Mechaniczny',
       'Wydział Mechaniczno-Energetyczny', 'Wydział Matematyki',
       'Wydział Informatyki i Zarządzania',
       'Wydział Elektroniki Mikrosystemów i Fotoniki'], dtype=object)

In [223]:
di = {'Wydział Architektury': 'W01', 'Wydział Budownictwa Lądowego i Wodnego': 'W02',
      'Wydział Chemiczny': 'W03', 'Wydział Elektroniki': 'W04',
      'Wydział Elektryczny': 'W05', 'Wydział Geoinżynierii, Górnictwa i Geologii' : 'W06',
      'Wydział Inżynierii Środowiska' : 'W07', 'Wydział Informatyki i Zarządzania' : 'W08',
      'Wydział Mechaniczno-Energetyczny' : 'W09', 'Wydział Mechaniczny' : 'W10',
      'Wydział Podstawowych Problemów Techniki': 'W11', 'Wydział Elektroniki Mikrosystemów i Fotoniki' : 'W12',
      'Wydział Matematyki': 'W13'}

df = df.replace({"wydzial": di})

In [227]:
df.to_csv("data/names_and_departaments.csv", index=False)